In [ ]:
from IPython.display import Image
import itertools

import numpy as np
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline

## Inference Networks with Adaptive Computation

This notebook outlines an idea for using RNN's [with adaptive computation](https://arxiv.org/abs/1603.08983) to produce [mixture posteriors](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.48.5846&rep=rep1&type=pdf) for [Density networks](http://www.inference.org.uk/mackay/ch_learning.pdf).

### 1.  Variational Inference with Mixture Approximations

Recall the method of variational inference.  For an approximation--$q(\boldsymbol{\theta};\boldsymbol{\phi})$--we fit it using the [*Evidence Lower Bound* (ELBO)](http://www.umiacs.umd.edu/~xyang35/files/understanding-variational-lower.pdf): $$ \log p(x) = \mathbb{E}_{q(\boldsymbol{\theta})}[ \log p(\mathbf{X}, \boldsymbol{\theta})] + \mathbb{H}_{q}[\boldsymbol{\theta}] + \text{KLD}[q(\boldsymbol{\theta}) || p(\boldsymbol{\theta} | \mathbf{X})] \ge \mathbb{E}_{q(\boldsymbol{\theta})}[ \log p(\mathbf{X}, \boldsymbol{\theta})] + \mathbb{H}_{q}[\boldsymbol{\theta}].$$

Often the posteriors we encounter in the wild are multi-modal, and thus using uni-modal posterior approximations can drastically misrepresent the posterior's support.  A straight-forward extentension is to use mixture approximations of the form: $$ q(\boldsymbol{\theta}) = \sum_{k} \pi_{k} q_{k}(\boldsymbol{\theta}) $$ where $\pi_{k}$ is a mixture weight and $q_{k}$ is the kth component density.  Using this approximation, the ELBO can be derived as $$ \log p(\mathbf{X}) \ge \sum_{k} \pi_{k} \mathbb{E}_{q_{k}(\boldsymbol{\theta})}[ \log p(\mathbf{X}, \boldsymbol{\theta})] + \mathbb{H}_{q}[\boldsymbol{\theta}] $$ where $\mathbb{H}_{q}[\boldsymbol{\theta}] =  -\sum_{k} \pi_{k} \int_{\boldsymbol{\theta}} q_{k}(\boldsymbol{\theta}) \log \sum_{j} \pi_{j} q_{j}(\boldsymbol{\theta}) \ d \boldsymbol{\theta}$, the entropy of the mixture model.  Unfortunately, this quantity is intractable, but we can use the following lower bound proposed by [Gershman et al. (2012)](https://arxiv.org/pdf/1206.4665.pdf): $$ -\sum_{k} \pi_{k} \int_{\boldsymbol{\theta}} q_{k}(\boldsymbol{\theta}) \log \sum_{j} \pi_{j} q_{j}(\boldsymbol{\theta}) \ d \boldsymbol{\theta} \ge -\sum_{k} \pi_{k}  \log \sum_{j} \pi_{j} \int_{\boldsymbol{\theta}} q_{k}(\boldsymbol{\theta}) q_{j}(\boldsymbol{\theta}) \ d \boldsymbol{\theta}, $$ which is attained via straight forward application of [Jensen's inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality).  For most common densities, the convolution above can be solved in closed form.  For Gaussians, we have: $$ \int_{\boldsymbol{\theta}} q_{k}(\boldsymbol{\theta}) q_{j}(\boldsymbol{\theta}) \ d \boldsymbol{\theta} = \mathbb{E}_{q_{k}}[q_{j}] = \text{N}(\boldsymbol{\mu}_{k}; \boldsymbol{\mu}_{j}, (\boldsymbol{\sigma}_{k}^{2} + \boldsymbol{\sigma}_{j}^{2})\mathbb{I} ).$$  Intuitively, the convolution term (entropy lower bound) can be seen as encouraging the components to have low probability under one another.  Or in other words, to spread out and diversify their posterior coverage.  Plugging the Gaussian solution back into the ELBO lower bound we have: $$\log p(\mathbf{X}) \ge \mathcal{L}_{\text{ELBO}} \ge   \sum_{k} \pi_{k} \mathbb{E}_{q_{k}(\boldsymbol{\theta})}[ \log p(\mathbf{X}, \boldsymbol{\theta})] + -\sum_{k} \pi_{k}  \log \sum_{j} \pi_{j} \text{N}(\boldsymbol{\mu}_{k}; \boldsymbol{\mu}_{j}, (\boldsymbol{\sigma}_{k}^{2} + \boldsymbol{\sigma}_{j}^{2})\mathbb{I} ).$$ 


### 2.  Adaptive Inference with RNNs

When using models with per-data-point latent variables of the form $$ \mathbf{z}_{i} \sim p(\mathbf{z}), \ \ \mathbf{x}_{i} \sim p(\mathbf{x}_{i} | \mathbf{z}_{i}),$$ it may be useful to have per-data-point posteriors $q(\mathbf{z}_{i} | \mathbf{x}_{i})$ of varying complexity.  For example, some data points may look like a blend of two different types (for example, a digit that looks like both a one and a seven) and we may wish to place posterior mass on the region of latent space for each digit.  We can define inference networks with this behavior by using an adaptive computation RNN as the inference network.  We propose a network with the following form: $$ \mathbf{h}_{i, t=k} = f_{\text{LSTM}}(\mathbf{h}_{i, t=k-1}, \mathbf{x}_{i}), \ \ \  \boldsymbol{\mu}_{i,k} = f_{\mu}(\mathbf{h}_{i,t=k}), \ \ \  \boldsymbol{\sigma}_{i,k} = f_{\sigma}(\mathbf{h}_{i,t=k}), \ \ \ \gamma_{i,k} = f_{\gamma}(\mathbf{h}_{i,t=k}).$$  The first variable $\mathbf{h}_{i,t=k}$ is simply the RNN's hidden state at time step $k$, as generated by an LSTM unit taking the previous hidden state $\mathbf{h}_{i,t=k-1}$ and the data point $\mathbf{x}_{i}$ as input.  The last three variables are components of the variational mixture approximation.  The first two--$\boldsymbol{\mu}_{i,k}$ and $\boldsymbol{\sigma}_{i,k}$--are the parameters of the kth Gaussian component and are computed via independent functions of the current hidden state.  The last variable is $\gamma_{i,k}$, taking on a value $(0,1)$, and thus would have a logistic function output.  We compose the $\gamma_{i,k}$ variables into the mixture components by using the Dirichlet's [stick breaking construction](https://en.wikipedia.org/wiki/Dirichlet_process#The_stick-breaking_process): $$ \pi_{i,1} = \gamma_{i,1}, \ \ \  \pi_{i,k} = \gamma_{i,k} \prod_{j=1}^{k-1} (1-\gamma_{i,j}).$$  The RNN will hault computation when the remaining stick is less than some threshold, i.e. $\prod_{k=1}^{K} (1-\gamma_{k}) < \epsilon$.  Following [Occam's razor](https://en.wikipedia.org/wiki/Occam%27s_razor), we assume mixtures with few components are preferred, and thus penalize the RNN's computation according to entropy of the mixture weights: $$\mathbb{H}[\boldsymbol{\pi}_{i}] = -\sum_{k=1}^{K} \pi_{i,k} \log \pi_{i,k} = -\sum_{k=1}^{K} \gamma_{i,k} \prod_{j=1}^{k-1} (1-\gamma_{i,j}) [ \log \gamma_{i,k} + \sum_{j=1}^{k-1} \log(1 - \gamma_{i,j}) ].$$

Plugging this term back into the ELBO lower bound in part #1, we have our final objective for the $i$th data point: $$ \mathcal{L}_{\text{ADAPT}}(\mathbf{x}_{i}) = \sum_{k} \pi_{i,k} \mathbb{E}_{q_{k}(\mathbf{z}_{i} | \mathbf{x}_{i})}[ \log p(\mathbf{x}_{i}, \mathbf{z}_{i})] - \sum_{k} \pi_{i,k}  \log \sum_{j} \pi_{i,j} \text{N}(\boldsymbol{\mu}_{i,k}; \boldsymbol{\mu}_{i,j}, (\boldsymbol{\sigma}_{i,k}^{2} + \boldsymbol{\sigma}_{i,j}^{2})\mathbb{I} ) - \mathbb{H}[\boldsymbol{\pi}_{i}].$$

### 3.  Toy Experiment: Approximating a Mixture

We will now implement the above ideas for a toy example in which we approximate a mixture density.  Let's first define a simple mixture...

In [ ]:
# Gaussian
def log_normal_pdf(x, mu, sigma):
    d = mu - x
    d2 = tf.mul(-1., tf.mul(d,d))
    s2 = tf.mul(2., tf.mul(sigma,sigma))
    return tf.reduce_sum(tf.div(d2,s2) - tf.log(tf.mul(sigma, 2.506628)))

# Gaussian Mixture
def log_gaussMix_pdf(z, pi, mu, sigma, K):
    s = tf.mul(pi[0], tf.exp(log_normal_pdf(z, mu[0], sigma[0])))
    for k in xrange(K-1):
        s += tf.mul(pi[k+1], tf.exp(log_normal_pdf(z, mu[k+1], sigma[k+1])))
    return tf.log(s)

And plot it...

In [ ]:
#'true' posterior params
mu_post = [-4., 3.]
sigma_post = [1., 3.]
pi_post = [.3, .7]

Z = tf.placeholder(tf.float32, shape=(), name="init")
log_prob_z = log_gaussMix_pdf(Z, pi_post, mu_post, sigma_post, 2)

# get posterior probabilities
z_grid = np.linspace(-10., 10., 1000).astype('float32')
with tf.Session() as session:
    probs_true = np.exp([session.run(log_prob_z, {Z: z}) for z in z_grid])

plt.figure()

plt.plot(z_grid, probs_true, 'b-', linewidth=5, label="True Posterior")

plt.xlabel("Z")
plt.xlim([-10,10])
plt.ylim([0,.25])
plt.legend()

plt.show()

Now let's define our adaptive inference RNN...

In [ ]:
# define dimensionality of model components
input_d = 1
rnn_hidden_d = 5
z_space_d = 1

# init data variable
X = tf.placeholder("float", [None, input_d])

# init RNN params
params = {'h':{'W':tf.Variable(tf.random_normal([input_d, rnn_hidden_d], stddev=.001)),
               'U':tf.Variable(tf.random_normal([rnn_hidden_d, rnn_hidden_d], stddev=.001)),
               'b':tf.Variable(tf.zeros([1, rnn_hidden_d]))
              },
          'mu':{'W':tf.Variable(tf.random_normal([rnn_hidden_d, z_space_d], stddev=.001)),
               'b':tf.Variable(tf.zeros([1, z_space_d]))
              },
          'sigma':{'W':tf.Variable(tf.random_normal([rnn_hidden_d, z_space_d], stddev=.001)),
               'b':tf.Variable(tf.zeros([1, z_space_d]))
              },
          'pi':{'W':tf.Variable(tf.random_normal([rnn_hidden_d, 1], stddev=.001)),
               'b':tf.Variable(tf.zeros([1, 1]))
              }
        }


# define variables for recursion
mu = tf.zeros([1, z_space_d])
sigma = tf.zeros([1, z_space_d])
pi = tf.zeros([1, 1])
hidden_state = tf.ones([1, rnn_hidden_d])
remaining_stick = tf.ones([1, 1])

# make lists to track intermediate values
mus = []
sigmas = []
pis = []


def prop_RNN(mu, sigma, pi, hidden_state, remaining_stick):

    # Defines an Elman network
    # TODO: define LSTM cell
    hidden_state = \
    tf.nn.relu(tf.matmul(X, params['h']['W']) + tf.matmul(hidden_state, params['h']['U']) + params['h']['b'])

    # Output parameters of approximation
    mu = tf.matmul(hidden_state, params['mu']['W']) + params['mu']['b']
    sigma = tf.nn.softplus(tf.matmul(hidden_state, params['sigma']['W']) + params['sigma']['b'])
    gamma = tf.nn.sigmoid(tf.matmul(hidden_state, params['pi']['W']) + params['pi']['b'])
    
    # Create pi_k via stick-breaking construction
    pi = gamma * remaining_stick
    remaining_stick = (1.-gamma) * remaining_stick
    
    # append to trackers, while_loop outputs only final values
    mus.append(mu)
    sigmas.append(sigma)
    pis.append(pi)
    
    return mu, sigma, pi, hidden_state, remaining_stick


stick_eps = .01
def test_stick(mu, sigma, pi, hidden_state, remaining_stick):
    return tf.reduce_sum(remaining_stick) > stick_eps


mu, sigma, pi, hidden_state, remaining_stick = tf.while_loop(cond=test_stick, \
                                                            body=prop_RNN, \
                                                            loop_vars=[mu, sigma, pi, hidden_state, remaining_stick])

Now let's define our optimization function...

In [ ]:
# data term: \sum_k \pi_k E[log p(x,z)]
expected_ll = 0.
for k, pi in enumerate(pis):
    z_sample = mus[k] + sigmas[k] * tf.random_normal(shape=[1, z_space_d])
    expected_ll += pi * log_gaussMix_pdf(z_sample, pi_post, mu_post, sigma_post, 2)
    
# entropy lower bound term : \sum_k pi_k log \sum_j pi_j N(mu_k; mu_j, sigma_k**2 + sigma_j**2)
ent_lb_term = 0.
for k, pi_k in enumerate(pis):
    temp_val = 0.
    for j, pi_j in enumerate(pis):
        temp_val += pi_j * tf.exp(log_normal_pdf(mus[k], mus[j], tf.sqrt(sigmas[k]**2 + sigmas[j]**2)))
    ent_lb_term += -pi_k * tf.log(temp_val)
    
# entropy of mixture weights
ent_mix_weights = 0.
for pi in pis:
    ent_mix_weights += -pi * tf.log(pi)
    
objective_fn = tf.reduce_sum(expected_ll) + tf.reduce_sum(ent_lb_term) - tf.reduce_sum(ent_mix_weights)

Lastly, let's train our dynamic RNN approximation...

In [ ]:
# Set training params
n_epochs = 20
learning_rate = .03

# get the training operator
train_model = tf.train.AdamOptimizer(learning_rate).minimize(-objective_fn, \
            var_list=[params['h']['W'], params['h']['U'], params['h']['b'], 
                     params['mu']['W'], params['mu']['b'],
                      params['sigma']['W'], params['sigma']['b'],
                      params['pi']['W'], params['pi']['b']])


with tf.Session() as session:
    tf.initialize_all_variables().run()
    
    for epoch_idx in xrange(n_epochs):
        
        # perform update
        _, loss = session.run([train_model, objective_fn], feed_dict={X: np.ones((1,1))})
        print "Epoch %d.  VI Objective: %.3f" %(epoch_idx, loss)